In [1]:

import config
import json
import os
import sys
import uuid
import datetime
import glob
import time
import uuid
import csv
import gradio as gr

from azure.identity import DefaultAzureCredential






from openai import AzureOpenAI
from azure.core.exceptions import AzureError
from azure.cosmos import CosmosClient, PartitionKey
from dotenv import dotenv_values
from azure.cosmos import ThroughputProperties
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# specify the name of the .env file name 
env_name = "argus.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)
# Azure Cosmos DB connection details
HOST = config['cosmos_host']
MASTER_KEY = config['cosmos_key']

cosmos_connection_string = config['cosmos_string']

container_name = "ChatMessages"


# Azure OpenAI connection details
openai_endpoint = config['openai_endpoint']
openai_key = config['openai_key']
openai_version = config['openai_version']
openai_embeddings_model = config['openai_embeddings_deployment']
openai_chat_model = config['AZURE_OPENAI_CHAT_MODEL']

# dalle = config['Dall-e']  

dbsource = config['cosmosdbsourcedb'] 
colvector = config['cosmosdbsourcecol']
cachecol = config['cosmsodbcache']
cosmosdbcolcompletion = config['cosmosdbcolcompletion']
container_name = config['cosmosdbcolcompletion']

# Create the OpenAI client
openai_client = AzureOpenAI(
  api_key = openai_key,  
  api_version = openai_version,  
  azure_endpoint =openai_endpoint 
)



#ENDPOINT =  config['cosmos_host']
#credential = DefaultAzureCredential()

#client = CosmosClient(ENDPOINT, credential)


In [2]:
def createvectordb(collection):
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)
    
    print(collection)
    
    vector_embedding_policy = { "vectorEmbeddings": [ {  "path": "/embedding",  "dataType": "float32",  "distanceFunction": "cosine",  "dimensions": 1536  } ] }
    indexing_policy = { "includedPaths": [ { "path": "/*" } ], "excludedPaths": [  {  "path": "/\"_etag\"/?" }  ], "vectorIndexes": [ {"path": "/embedding", "type": "diskANN"  }] }

    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id'), 
        indexing_policy=indexing_policy, 
        vector_embedding_policy=vector_embedding_policy) 
        print("container created")
              
    except : 
         raise

In [3]:
from azure.cosmos import exceptions, PartitionKey


client = CosmosClient(HOST, {'masterKey': MASTER_KEY})

mydbt = client.create_database_if_not_exists(dbsource)

mydbt = client.get_database_client(dbsource)
    
# Create the vector embedding policy to specify vector details
vector_embedding_policy = {
    "vectorEmbeddings": [ 
        { 
            "path":"/vector" ,
             "dataType":"float32",
            "distanceFunction":"dotproduct",
            "dimensions":1536
        }, 
    ]
}

# Create the vector index policy to specify vector details
indexing_policy = {
    "vectorIndexes": [ 
        {
            "path": "/vector", 
            "type": "diskANN" 
        }
    ]
} 





# Create the cache collection with vector index
try:
    cache_container =  mydbt.create_container_if_not_exists( id=cachecol, 
                                                  partition_key=PartitionKey(path='/id'), 
                                                  indexing_policy=indexing_policy,
                                                  vector_embedding_policy=vector_embedding_policy
                                                  ) 
    print('Container with id \'{0}\' created'.format(id)) 

except exceptions.CosmosHttpResponseError: 
        raise 

Container with id '<built-in function id>' created


In [4]:
def generate_embeddings(openai_client, text):
    """
    Generates embeddings for a given text using the OpenAI API v1.x
    """
    
    print("Generating embeddings for: ", text, " with model: ", openai_embeddings_model)
    response = openai_client.embeddings.create(
        input = text,
        model= openai_embeddings_model
        
    )
    
    embeddings = response.data[0].embedding
    return embeddings

In [22]:
def loaddata(db,collection) :
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
  
    print("Creating container..."   )
    try:
        container = mydbt.create_container_if_not_exists( 
        id= collection, 
        partition_key=PartitionKey(path='/id')
        )
        query = "SELECT * FROM c"
        source = mydbt.get_container_client("documents")
        result = source.query_items(
            query=query,
            enable_cross_partition_query=True)

        for item in result:
            item['text']= json.dumps(item)
            container.upsert_item(item)

    
        
# count products
        query = "SELECT VALUE COUNT(1) FROM c"

        total_count = 0

        result = container.query_items(
            query=query,
            enable_cross_partition_query=True)

        for item in result:
            total_count += item

        print("Total count:", total_count)
        
       
    
    except : 
     raise  
       

  

In [5]:
def add_doc(openai_client, collection, doc,name):
   
  
  
    try:
        doc1 = {}
        doc1["id"] = doc["id"]
        doc1["source"]= name
        doc1["embedding"] = generate_embeddings(openai_client, json.dumps(doc))
        
        print(doc["id"])
        
        collection.upsert_item(doc1)
       
    except Exception as e:
        print(str(e))

In [6]:

def get_completion(openai_client, model, prompt: str):    
   
    print("Prompt: ", prompt)
   
    response = openai_client.chat.completions.create(
        model = model,
        messages =   prompt,
        temperature = 0.1
    )   
   
    return response.model_dump()


In [7]:
def get_similar_docs(openai_client, db, query_text, limit):
    """ 
        Get similar documents from Cosmos DB for NoSQL 

        input: 
            container: name of the container
            query_text: user question
            limit: max number of documents to return
        output:
            documents: json documents similar to the user question
            elapsed_time
    """
    # vectorize the question
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(db)   
    cvector = mydbt.get_container_client(colvector)
    sim =0.78
   
    query_vector = generate_embeddings(openai_client, query_text)
  
     
     
    query = f"""
        SELECT TOP @num_results  c.id,c.source, VectorDistance(c.embedding, @embedding) as SimilarityScore 
        FROM c
        WHERE VectorDistance(c.embedding,@embedding) > @similarity_score
        ORDER BY VectorDistance(c.embedding,@embedding)
    """
    results = cvector.query_items(
        query=query,
         parameters=[
            {"name": "@embedding", "value": query_vector},
            {"name": "@num_results", "value": limit},
            {"name": "@similarity_score", "value": sim}
        ],
        enable_cross_partition_query=True, populate_query_metrics=True
    )   
    
           
    listid = []
    source = ""
    # get products from list of id
    id_list = [id for id in results]

    for i in id_list:
            listid.append(i['id'])
            source = (i['source'])
                                  
            
        
        
 
    
    if listid == []:
        products = []
    else : 
        id_list_str = ', '.join([f"'{id}'" for id in listid]) 
        
      
        mycolt = mydbt.get_container_client(source)
            
        query = f"""
                    SELECT * FROM c 
                    WHERE  c.id IN ({id_list_str})
                """
                
            
        results = mycolt.query_items(
                    query=query,
                    enable_cross_partition_query=True
        )

        products = []
        for product in results:
            products.append(product)    

          
    

    return products
            

In [9]:
def ReadFeed(collection):
        
       
        
        
        client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
        mydbt = client.get_database_client(dbsource)   
        mycolt = mydbt.get_container_client(collection)
        mycoltembed = mydbt.get_container_client("vector") 
        name = collection
        
     
        # Define a point in time to start reading the feed from
        time = datetime.datetime.now()
        
        print (time)
        time = time - datetime.timedelta(days=1)
        print (time)
        
           
        
        response = mycolt.query_items_change_feed(start_time=time)
        #response = mycolt.query_items_change_feed( )
        
        for doc in response:
            add_doc(openai_client, mycoltembed, doc,name)
           

In [10]:
def get_chat_history( completions=3):
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)   
    container = mydbt.get_container_client(cachecol)
    
    results = container.query_items(
        query= '''
        SELECT TOP @completions *
        FROM c
        ORDER BY c._ts DESC
        ''',
        parameters=[
            {"name": "@completions", "value": completions},
        ], enable_cross_partition_query=True)
    results = list(results)
    return results

In [11]:
def cache_search( vectors, similarity_score = 0.99 , num_results=5):
    # Execute the query
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)   
    container = mydbt.get_container_client(cachecol)
    
    results = container.query_items(
        query= '''
        SELECT TOP @num_results  c.completion, VectorDistance(c.vector, @embedding) as SimilarityScore 
        FROM c
        WHERE VectorDistance(c.vector,@embedding) > @similarity_score 
        ORDER BY VectorDistance(c.vector,@embedding)
        ''',
        parameters=[
            {"name": "@embedding", "value": vectors},
            {"name": "@num_results", "value": num_results},
            {"name": "@similarity_score", "value": similarity_score}
        ],
        enable_cross_partition_query=True, populate_query_metrics=True)
   
    formatted_results = []
    for result in results:
        print("result query")
        print(result)
        formatted_results.append(result)

  
    return formatted_results

In [12]:
def cacheresponse(user_prompt, prompt_vectors, response):
    
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)   
    container = mydbt.get_container_client(cachecol)
    
    print("Caching response for prompt: ", user_prompt)
    print("Response: ", response)
    
    
    # Create a dictionary representing the chat document
    chat_document = {
        'id':  str(uuid.uuid4()),  
        'prompt': user_prompt,
        'completion': response['choices'][0]['message']['content'],
        'completionTokens': str(response['usage']['completion_tokens']),
        'promptTokens': str(response['usage']['prompt_tokens']),
        'totalTokens': str(response['usage']['total_tokens']),
        'model': response['model'],
        'vector': prompt_vectors
    }
    # Insert the chat document into the Cosmos DB container
    container.create_item(body=chat_document)
    print("item inserted into cache.", chat_document)

In [13]:
def clearcache ():
   
    client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
    mydbt = client.get_database_client(dbsource)   
  
    
      
# Create the vector embedding policy to specify vector details
    vector_embedding_policy = {
    "vectorEmbeddings": [ 
        { 
            "path":"/vector" ,
             "dataType":"float32",
            "distanceFunction":"cosine",
            "dimensions":1536
        }, 
    ]
}

# Create the vector index policy to specify vector details
    indexing_policy = { 
    "vectorIndexes": [ 
        {
            "path": "/vector", 
            "type": "diskANN"
            
        }
    ]
    } 
   
    mydbt.delete_container(cachecol)


# Create the cache collection with vector index
    try:
        mydbt.create_container_if_not_exists( id=cachecol, 
                                                  partition_key=PartitionKey(path='/id'), 
                                                  indexing_policy=indexing_policy,
                                                  vector_embedding_policy=vector_embedding_policy
                                                ) 
        print('Container with id \'{0}\' created'.format(id)) 

    except exceptions.CosmosHttpResponseError: 
        raise 
    

In [14]:
def generatecompletionede(user_prompt, vector_search_results, chat_history):
    
    system_prompt = '''
    You are an intelligent assistant for ypurdata . You are designed to provide helpful answers to user questions about  your data.
    You are friendly, helpful, and informative and can be lighthearted. Be concise in your responses, but still friendly.
        - Only answer questions related to the information provided below. 
        - Write two lines of whitespace between each answer in the list.
    '''

    # Create a list of messages as a payload to send to the OpenAI Completions API

    # system prompt
    
    messages = [{'role': 'system', 'content': system_prompt}]
    
    #chat history
    for chat in chat_history:
        messages.append({'role': 'user', 'content': chat['prompt'] + " " + chat['completion']})
    
    #user prompt
    messages.append({'role': 'user', 'content': user_prompt})

    #vector search results
    for result in vector_search_results:
        messages.append({'role': 'system', 'content': result['text']})

    
    # Create the completion
    response = get_completion(openai_client, openai_chat_model, messages)
    print("Response from openai", response)
    
    return response

In [15]:
def chat_completion(user_input):

    # Generate embeddings from the user input
    user_embeddings = generate_embeddings(openai_client, user_input)
    
    # Query the chat history cache first to see if this question has been asked before
    cache_results = cache_search(vectors=user_embeddings , num_results=1)

    if len(cache_results) > 0:
        return cache_results[0]['completion'], True
    else:
        # Perform vector search on the movie collection
        print("\n New result\n")
        search_results = get_similar_docs(openai_client, dbsource, user_input, 3)
        print("\n search result done\n")
        
        
        print("Getting Chat History\n")
        # Chat history
        chat_history = get_chat_history(1)

        # Generate the completion
        print("Generating completions \n")
        completions_results = generatecompletionede(user_input, search_results, chat_history)

        print("Caching response \n")
        # Cache the response
        cacheresponse(user_input, user_embeddings, completions_results)

        print("\n")
        # Return the generated LLM completion
        return completions_results['choices'][0]['message']['content'], False

In [24]:
#createvectordb(colvector)
loaddata(dbsource,'argus')
ReadFeed('argus')

In [25]:



chat_history = []
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat genereric with your data ")
     
    msg = gr.Textbox(label="Ask me about Your  Data!")
    clear = gr.Button("Clear")

    def user(user_message, chat_history, pk):
        # Create a timer to measure the time it takes to complete the request
        start_time = time.time()
        # Get LLM completion
        response_payload, cached = chat_completion(user_message)
                
        
        # Stop the timer
        end_time = time.time()
        elapsed_time = round((end_time - start_time) * 1000, 2)
        response = response_payload
        # Append user message and response to chat history
        details = f"\n (Time: {elapsed_time}ms)"
        if cached:
            details += " (Cached)"
        chat_history.append([user_message, response_payload + details])
        
        return gr.update(value=""), chat_history
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)

    clear.click(lambda: None, None, chatbot, queue=False)

c:\Users\edeletang\Documents\demonosqlcomsosdb\.venv\Lib\site-packages\gradio\utils.py:986: UserWarning: Expected 3 arguments for function <function user at 0x0000016EBA9C6A20>, received 2.
  warnings.warn(
c:\Users\edeletang\Documents\demonosqlcomsosdb\.venv\Lib\site-packages\gradio\utils.py:990: UserWarning: Expected at least 3 arguments for function <function user at 0x0000016EBA9C6A20>, received 2.
  warnings.warn(


In [26]:
demo.launch(debug=True)

c:\Users\edeletang\Documents\demonosqlcomsosdb\.venv\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.39.0, however version 5.0.1 is available, please upgrade. 
--------
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\edeletang\Documents\demonosqlcomsosdb\.venv\Lib\site-packages\gradio\helpers.py:974: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


Generating embeddings for:  as t'on des informations sur AN PATRIMOINE SÉRÉNITÉ OBSÈQUES  with model:  text-embedding-ada-002

 New result

Generating embeddings for:  as t'on des informations sur AN PATRIMOINE SÉRÉNITÉ OBSÈQUES  with model:  text-embedding-ada-002

 search result done

Getting Chat History

Generating completions 

Prompt:  [{'role': 'system', 'content': '\n    You are an intelligent assistant for ypurdata . You are designed to provide helpful answers to user questions about  your data.\n    You are friendly, helpful, and informative and can be lighthearted. Be concise in your responses, but still friendly.\n        - Only answer questions related to the information provided below. \n        - Write two lines of whitespace between each answer in the list.\n    '}, {'role': 'user', 'content': "as t'on des informations sur Clinica Rugani - Siena Oui, voici les informations disponibles sur Clinica Rugani - Siena :\n\n1. **Nom de la clinique**: Clinica Rugani - Siena\n\n2

In [ ]:
demo.close()